<a href="https://colab.research.google.com/github/R12942159/NTU_DLCV/blob/Hw4/p1_NeRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1hF4z9U-xaoV4qaq9DbhTP-KKJTlwOUv_ -O hw4_data.zip
!unzip hw4_data.zip
!rm hw4_data.zip

In [ ]:
!pip install kornia

In [11]:
import os
import glob
import json
import torch
import random
import numpy as np

from PIL import Image
from kornia import create_meshgrid
from scipy.spatial import transform
from torch.utils.data import Dataset
from torchvision import transforms as T

In [3]:
metadata_path = '/content/dataset/metadata.json'
output_floder = '/content'

In [4]:
with open(metadata_path, 'r') as file:
    metadata_json = json.load(file)

In [5]:
width = metadata_json['metadata']['width']
height = metadata_json['metadata']['height']

In [ ]:
metadata_json['metadata']
metadata_json['camera']
metadata_json['segmentation_labels']
metadata_json['scene_boundaries']
metadata_json['split_ids']

In [35]:
class NeRFDataset(torch.utils.data.Dataset):

    def __init__(self, metadata_path, dataset_path, mode='train'):
        super(NeRFDataset, self).__init__()
        self.dataset_path = dataset_path
        self.mode = mode

        with open(metadata_path, 'r') as file:
            self.metadata_json = json.load(file)

    def __len__(self) -> int:
        return len(metadata_json['split_ids'][self.mode])

    def __getitem__(self, idx):
        id = self.metadata_json['split_ids'][self.mode][idx]
        path = os.path.join(self.dataset_path, (str(id).rjust(5, '0') + '.jpg'))
        positions = self.metadata_json['camera']['positions'][id]
        quaternions = self.metadata_json['camera']['quaternions'][id]

        return {'id': id, 'path': path, 'positions': positions, 'quaternions': quaternions}

In [36]:
data = NeRFDataset(metadata_path, metadata_path)

In [39]:
for i in data:
    print(i['id'])
    break

166
